In [ ]:
import tensorflow as tf
from deepTAM.helpers2 import *
from deepTAM.common_netcode.myHelpers import *
gpu_options = tf.GPUOptions()
gpu_options.per_process_gpu_memory_fraction=0.8
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options))

# Option 1
saver = tf.train.import_meta_graph('/misc/lmbraid19/thomasa/deep-networks/deepTAM/nets_multiframes/depth_singleimage/training_v_sun3d/1_d_big/recovery_checkpoints/snapshot-391065.meta')
saver.restore(sess, '/misc/lmbraid19/thomasa/deep-networks/deepTAM/nets_multiframes/depth_singleimage/training_v_sun3d/1_d_big/recovery_checkpoints/snapshot-391065')

# Option 2 (Huizhong's way)
from deepTAM.evaluation.helpers import *
from tfutils import optimistic_restore
import cv2
import numpy as np

mapping_mod = load_myNetworks_module_noname('/misc/lmbraid19/thomasa/deep-networks/deepTAM/nets_multiframes/depth_singleimage/net/myNetworks.py')
single_image_depth_net = mapping_mod.SingleImageDepthNetwork(batch_size=1, width=320, height=240)
single_image_depth_outputs = single_image_depth_net.build_net(**single_image_depth_net.placeholders)
sess.run(tf.global_variables_initializer())
checkpoint = '/misc/lmbraid19/thomasa/deep-networks/deepTAM/nets_multiframes/depth_singleimage/training_v_sun3d/1_d_big/recovery_checkpoints/snapshot-391065'
optimistic_restore(sess,checkpoint,verbose=True)
	
rgb = cv2.imread('/misc/lmbraid19/thomasa/datasets/rgbd/rgbd_dataset_freiburg1_teddy/rgb/1305032227.675702.png', -1)
rgb_new = cv2.normalize(rgb, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
rgb_new = cv2.resize(rgb_new,(320, 240))
rgb_new = np.transpose(rgb_new, (2,0,1))

feed_dict = {
                single_image_depth_net.placeholders['image']: rgb_new[np.newaxis, :, :, :],
            }

single_image_depth_out = sess.run(single_image_depth_outputs, feed_dict=feed_dict)
depth_pr = single_image_depth_out['predict_depth']

cv2.imshow("predicted depth", depth_pr[0,0])
cv2.imshow("input image", rgb)
cv2.waitKey(0)

In [ ]:
import tensorflow as tf
from deepTAM.helpers2 import *
from deepTAM.common_netcode.myHelpers import *
from deepTAM.evaluation.helpers import *
from tfutils import optimistic_restore
import cv2
import numpy as np
from cv_bridge import CvBridge

In [ ]:
gpu_options = tf.GPUOptions()
gpu_options.per_process_gpu_memory_fraction=0.8
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options))


In [ ]:
mapping_mod = load_myNetworks_module_noname('/misc/lmbraid19/thomasa/deep-networks/deepTAM/nets_multiframes/depth_singleimage/net/myNetworks.py')
single_image_depth_net = mapping_mod.SingleImageDepthNetwork(batch_size=1, width=320, height=240)
single_image_depth_outputs = single_image_depth_net.build_net(**single_image_depth_net.placeholders)
sess.run(tf.global_variables_initializer())
checkpoint = '/misc/lmbraid19/thomasa/deep-networks/deepTAM/nets_multiframes/depth_singleimage/training_v_sun3d/1_d_big/recovery_checkpoints/snapshot-391065'
optimistic_restore(sess,checkpoint,verbose=True)


In [ ]:
rgb = cv2.imread('/misc/lmbraid19/thomasa/datasets/rgbd/rgbd_dataset_freiburg1_teddy/rgb/1305032227.675702.png', -1)
rgb_new = cv2.normalize(rgb, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
rgb_new = cv2.resize(rgb_new,(320, 240))
rgb_new = np.transpose(rgb_new, (2,0,1))


In [ ]:
feed_dict = {
                single_image_depth_net.placeholders['image']: rgb_new[np.newaxis, :, :, :],
            }

single_image_depth_out = sess.run(single_image_depth_outputs, feed_dict=feed_dict)

In [ ]:
depth_pr = single_image_depth_out['predict_depth'][0,0]


In [ ]:
_cv_bridge_depth = CvBridge()

In [ ]:
depthmap_predicted_msg = _cv_bridge_depth.cv2_to_imgmsg(depth_pr, "passthrough")

In [ ]:
depthmap_predicted_msg.step = int (depthmap_predicted_msg.step)
depthmap_predicted_msg.step